In [ ]:
%run /Utils/Functions/core

In [ ]:
import requests
import concurrent.futures
import time
import sys
import os
from time import sleep, mktime
from uuid import uuid4
import json
from requests.auth import HTTPBasicAuth
from datetime import datetime, timedelta, timezone
from IPython.display import clear_output

In [ ]:
## Cria as variáveis de LOG
format_log = '%Y-%m-%d %H:%M:%S'
dtInicio = datetime.today() - timedelta(hours=3)
dtInicio_format = dtInicio.strftime(format_log)
tipo_log = 'API'
camada = 'landing'
emissor = '<org>'
atividade = '<activity_desc>'
origem = 'RestService'
destino = 'AzureBlobFS'
execUrl = ' '

try:
    infos = json.loads(dbutils.notebook.entry_point.getDbutils().notebook().getContext().toJson()) # captura as informações do job que executa o notebook
    orgId = infos['tags']['orgId']
    runId = infos['tags']['multitaskParentRunId']
    jobId = infos['tags']['jobId']
    if orgId == '2960871991268730': # Monta a URL caso seja o ID do ambiente de DEV
        execUrl = f'https://adb-{orgId}.10.azuredatabricks.net/?o={orgId}#job/{jobId}/run/{runId}' # cria a url de execução do 
    else: # Monta a URL caso seja o ID do ambiente de PROD
        execUrl = f'https://adb-{orgId}.15.azuredatabricks.net/?o={orgId}#job/{jobId}/run/{runId}' # cria a url de execução do 
except:
    print('Campo de URL não pode ser identificado!')

In [ ]:
%sql
use catalog prod;

In [ ]:
dbutils.widgets.text("dt_ingestao", "")

dt_ingestao = getArgument("dt_ingestao").upper().strip()

location_landing = spark.sql("show external locations").select("url").where("name = 'landing-area'").collect()[0][0]
location_flat_file = spark.sql("show external locations").select("url").where("name = 'flatfile-area'").collect()[0][0]

In [ ]:
# Formata o dt_ingestao
format_timestamp = '%Y-%m-%d %H:%M:%S.%f'
dt_ingestao = datetime.now() if dt_ingestao == "" else datetime.strptime(dt_ingestao, format_timestamp)

# Pega o horário atual e tira 3 horas para converter para BRT (UTC -3)
# Tira 5 minutos por conta de um limite da API
dt_fim = dt_ingestao - timedelta(hours=3) - timedelta(minutes=5)
dt_inicio = dt_fim - timedelta(days=1)

# Força as horas, minutos e segundos em 0
dt_inicio = datetime(dt_inicio.year, dt_inicio.month, dt_inicio.day, 0, 0, 0)

# Separa as datas no formato UNIX
dt_fim_unix = int(mktime(dt_fim.timetuple()))
dt_inicio_unix = int(mktime(dt_inicio.timetuple()))

# dt_inicio_unix = 1546300800

# Transforma as datas em STRING utilizando um formato de timestamp
dt_fim = dt_fim.strftime(format_timestamp)
dt_inicio = dt_inicio.strftime(format_timestamp)

print(f"dt_inicio: {dt_inicio} | unix: {dt_inicio_unix}")
print(f"dt_fim   : {dt_fim} | unix: {dt_fim_unix}")

In [ ]:
username = dbutils.secrets.get("scope-vault-data", "zendesk-user")
password = dbutils.secrets.get("scope-vault-data", "zendesk-pass")

domain = "zendesk"
subdomain = "<subdomain>"
threads = 10
baseUrl = f"https://{subdomain}.{domain}.com"

dir_flatfile = f'{location_flat_file}/<org>/zendesk'
dir_flatfile_ro = f'/dbfs/FileSotre/flat-files/.../<org>/zendesk'

org_memberships_path = dir_flatfile+"/organization_memberships/last-page-org-memberships.txt"
org_memberships_path_ro = dir_flatfile_ro+"/organization_memberships/last-page-org-memberships.txt"

print(baseUrl)

In [ ]:
# Pega apenas cargas que possuem valor preenchido no campo "ds_Url" para não trazer tabelas que são geradas a partir de outro evento na landing.
df_param = fn_ConsultaJdbc("""
    SELECT pca.*
    FROM ctl.ADF_Parametro_Carga_API pca
    WHERE pca.fl_ativo = 1
    and nm_Sistema = 'zendesk'
    and ds_Url is not null
""")

data_param = df_param.collect()

display(df_param)

In [ ]:
def get_tables(param, baseUrl, dt_inicio_unix, dtIngestao, location_landing):
    end_point_page = None
    try:
        format_timestamp = '%Y-%m-%d %H:%M:%S.%f'
        dtIngestao = str(dtIngestao)
        if param.vl_Ultimo_Incremento is not None:
            vlUltimoIncremento = datetime.strptime(param.vl_Ultimo_Incremento, format_timestamp)
            vlUltimoIncremento = int(mktime(vlUltimoIncremento.timetuple()))
        else:
            vlUltimoIncremento = dt_inicio_unix

        vlUltimoIncremento = str(vlUltimoIncremento)
        # vlUltimoIncremento = str(1685577600)

        # Declarando as variáveis
        pagina = 1
        max_errors = 100
        error_sleep = 60
        inicio = time.time()

        # Variáveis utilizadas apenas por algumas cargas, inicia com valores iguais para não impactar as tabelas que não utilizam
        count = -1
        count_api = -1

        # Landing
        if param.vl_Schedule_Carga != 0:
            dir_landing = f"{location_landing}/{param.ds_Diretorio_Landing}/{param.ds_Nome_Arquivo_Landing}/{dtIngestao[0:4]}/{dtIngestao[5:7]}/{dtIngestao[8:10]}/{dtIngestao[11:13]}".lower()
            alter_landing = f"{location_landing}/{param.ds_Diretorio_Landing}/@alter_table/{dtIngestao[0:4]}/{dtIngestao[5:7]}/{dtIngestao[8:10]}/{dtIngestao[11:13]}".lower()
        else:
            dir_landing = f"{location_landing}/{param.ds_Diretorio_Landing}/{param.ds_Nome_Arquivo_Landing}/{dtIngestao[0:4]}/{dtIngestao[5:7]}/{dtIngestao[8:10]}".lower()
            alter_landing = f"{location_landing}/{param.ds_Diretorio_Landing}/@alter_table/{dtIngestao[0:4]}/{dtIngestao[5:7]}/{dtIngestao[8:10]}".lower()
        
        ## ALGUNS ENDPOINTS NÃO SUPORTAM A PAGINAÇÃO POR CURSOR.    
        # PARA ATIVAR NOS QUE POSSUEM SUPORTE, É SÓ ADICIONAR "?page[size]=100" NO ENDPOINT
        # https://developer.zendesk.com/api-reference/introduction/pagination/
        print(f"Tabela {param.ds_Nome_Arquivo_Landing}! Começando carga...")

        # Cria um objeto de lista para salvar os dados dessa iteração.
        result_data = []

        # Pega o endpoint que utilizará no request
        # Verificação para casos especiais que salvam a última URL que processou
        if param.ds_Custom_Field is None:
            end_point_page = f"{baseUrl}/{param.ds_Url}".replace("@DT_INI_CARGA", vlUltimoIncremento)
        else:
            end_point_page = param.ds_Custom_Field

        # Envia a primeira requisição
        req = requests.get(end_point_page, auth=HTTPBasicAuth(f"{username}", f"{password}"))
        last_endpoint = end_point_page
        print(last_endpoint)

        # Verifica se a primeira requisição teve erro
        if req.status_code != 200:
            print("\n"+f"Tabela {param.ds_Nome_Arquivo_Landing} com erro na primeira requisição!")
        # Caso a response da requisição seja 200 (sucesso) 
        else:
            # Armazena o conteúdo da API na variável data em formato JSON
            data = req.json()
            data_keys = list(data.keys())

            end_point_page = None
            if 'meta' in data_keys:
                if 'has_more' in list(data['meta'].keys()):
                    if data['meta']['has_more'] == True:
                        try:
                            # Tenta capturar o campo "links: next" na API para consultar a próxima página, caso não conseguir, retorna erro
                            end_point_page = data['links']['next']
                        except:
                            raise Exception(f'Não foi possível encontrar a próxima página para a tabela --> {param.ds_Nome_Arquivo_Landing}\nPágina --> {pagina}')
            
            if 'next_page' in data_keys:
                if 'count' in data_keys:
                    if data['count'] < 50:
                        end_point_page = None
                    else:
                        end_point_page = data['next_page']
                
            if  'count' in data_keys:
                # Captura o campo "count" na API para verificar o tamanho padrão dos registros
                count_api = data['count']
                # Cria uma variável com o valor igual o da API que vai ser modificada a cada requisição obtendo o novo valor do campo "count"
                count = count_api

            # Caso tenha o campo end_of_steam, utiliza ele para definir o término do incremental
            if 'end_of_stream' in data_keys:
                end_stream = data['end_of_stream']
                print("End_stream", end_stream)
            else:
                end_stream = False

            # Algumas tabelas salvam a partir de colunas com o nome diferente da tabela
            if param.nm_Item_Origem in ('ticket_metrics'):
                result_data.extend(data['metric_sets'])
            else:
                result_data.extend(data[param.nm_Item_Origem])

            if len(result_data) == 0: # caso a quantidade de registros seja 0 (vazio), encerra a execução para evitar erros no LOG
                return

            # Atualiza a contagem de páginas lidas
            pagina += 1

            # Atualiza o ds_Custom_Field para tabelas que utilizam a coluna
            if param.ds_Custom_Field is not None:
                fn_AtualizaCustomField_API(param.id_Parametro_Carga_API, last_endpoint)

        # Faz requisições até que termine a paginação completa
        inicio_laco = time.time()
        while end_point_page is not None:
            if end_stream == True:
                # Caso o campo "end_of_stream" seja igual a True não percorre mais páginas
                break

            if count != count_api:
                # Faz requisições até que o campo "count" da API venha com valor diferente do padrão da primeira página, significa que todas as páginas foram percorridas
                break

            # Envia a requisição para a página seguinte
            req = requests.get(end_point_page, auth=HTTPBasicAuth(f"{username}", f"{password}"))
            last_endpoint = end_point_page

            # Em caso de erro entra no laço
            if req.status_code != 200:
                # Caso erro: printa o erro e a página, espera a quantidade de tempo declarada no "error_sleep" e confere se tiveram mais de 10 erros
                print("\n"+f"Tabela {param.nm_Item_Origem} com erro na requisição da página {pagina}")
                print(str(req.content))
                max_errors -= 1
                if(max_errors < 0):
                    raise Exception(str(req.content))

                sleep(error_sleep)
            # Caso a response da requisição seja 200 (sucesso): Salva o conteúdo na variável "data", captura o campo "next_page" e armazena todo o conteúdo na lista dinâmica
            else:
                data = req.json()
                data_keys = list(data.keys())

                # Verifica todas as situações de paginação
                end_point_page = None
                if 'meta' in data_keys:
                    if 'has_more' in list(data['meta'].keys()):
                        if data['meta']['has_more'] == True:
                            try:
                                # Tenta capturar o campo "links: next" na API para consultar a próxima página, caso não conseguir, retorna erro
                                end_point_page = data['links']['next']
                            except:
                                raise Exception(f'Não foi possível encontrar a próxima página para a tabela --> {param.ds_Nome_Arquivo_Landing}\nPágina --> {pagina}')
                
                if 'next_page' in data_keys:
                    # Tenta capturar o campo "next_page" na API para verificar se tem mais de uma página, caso não, considera o "next_page" igual a Null
                    end_point_page = data['next_page']

                if 'count' in data_keys:
                    # Captura o campo "count" na API para verificar o tamanho padrão dos registros
                    count_api = data['count']
                    print("Count...", count_api, count)

                # Caso tenha o campo end_of_steam, utiliza ele para definir o término do incremental
                if 'end_of_stream' in data_keys:
                    end_stream = data['end_of_stream']
                    print("End_stream", end_stream)
                else:
                    end_stream = False

                # Algumas tabelas salvam a partir de colunas com o nome diferente da tabela
                if param.nm_Item_Origem in ('ticket_metrics'):
                    result_data.extend(data['metric_sets'])
                else:
                    result_data.extend(data[param.nm_Item_Origem])
                    print(f"Tabela {param.nm_Item_Origem} ---> Página {pagina} > {len(data[param.nm_Item_Origem])} registros")

                # Atualiza a contagem de páginas lidas
                pagina+=1
                print("Páginas processadas:", pagina)
                # Atualiza o ds_Custom_Field para tabelas que utilizam a coluna
                if param.ds_Custom_Field is not None:
                    fn_AtualizaCustomField_API(param.id_Parametro_Carga_API, last_endpoint)

            fim_laco = time.time()
            if (fim_laco - inicio_laco) > 7200:
                raise Exception(f'A quantidade de tempo determinada foi excedida!\nErro na tabela full --> {param.ds_Nome_Arquivo_Landing}\nPágina --> {pagina}')

        fim = time.time()
        
        # Subtrai a variável "início" pela "fim" para obter o tempo total de execução da tabela e armazena na váriavel "tempo_exec"
        tempo_exec = fim - inicio

        print("\n"+f"A tabela {param.nm_Item_Origem} teve {pagina} páginas carregadas com êxito!")
        print(f"Tamanho total de registros {len(result_data)}")
        print(f"O tempo de execução da tabela {param.nm_Item_Origem} foi {tempo_exec}")

        # PROCESSO DE CARGA DOS DADOS NO BLOB
        fn_SaveJson(result_data, dir_landing, str(param.ds_Nome_Arquivo_Landing).lower())

        fn_AtualizaUltimoIncremento_API(param.id_Parametro_Carga_API, dtIngestao)

        ## Constrói novos eventos a partir do evento principal da requisição da API

        # Tabela Child_Events
        if param.nm_Item_Origem == 'ticket_events':
            result_child_events  = []
            for data in result_data:
                event_child = data['child_events']
                for i in range(len(event_child)):
                    compo_event_child = event_child[i]
                    compo_event_child['ticket_id'] = data['ticket_id']
                    compo_event_child['created_at'] = data['created_at']
                    result_child_events.append(compo_event_child)
            
            print("Criando a tabela child_events...")
            fn_SaveJson(result_child_events, alter_landing.replace("@alter_table", "child_events"), "child_events")
            fn_AtualizaUltimoIncremento_API(314, dtIngestao)

        # Tabela custom_field_options
        if param.nm_Item_Origem == 'ticket_fields':
            result_field_options = []
            for data in result_data:
                if 'custom_field_options' in list(data.keys()):
                    field_options = data['custom_field_options']
                    for i in range(len(field_options)):
                        compo_field_options = field_options[i]
                        compo_field_options['fields_id'] = data['id']
                        result_field_options.append(compo_field_options)

            print("Criando a tabela custom_field_options...")
            fn_SaveJson(result_field_options, alter_landing.replace("@alter_table", "custom_field_options"), "custom_field_options")
            fn_AtualizaUltimoIncremento_API(316, dtIngestao)
    
        ## LOG SUCESSO
        dtFim = datetime.today() - timedelta(hours=3)
        dtFim_format = dtFim.strftime(format_log)
        duracao = int((dtFim-dtInicio).total_seconds()) #captura a duração subtraindo o dtFim pelo dtInicio
        dsParametro = str(param.asDict()) #captura todos os parâmetros
        notebook = dbutils.notebook.entry_point.getDbutils().notebook().getContext().notebookPath().get()
        
        try:
            pipeline = param.nm_Pipeline
        except:
            pipeline = os.path.basename(notebook)

        if end_point_page is not None:
            query = end_point_page
        else:
            query = ' '
            
        parametros = {"tipo_log": tipo_log,"id_parametro": param.id_Parametro_Carga_API, "camada": camada, "dtInicio": dtInicio_format, "dtFim": dtFim_format, "pipeline": pipeline, "atividade": atividade, "notebook": notebook, "origem": origem, "destino": destino, "sistema": param.nm_Sistema, "emissor": emissor, "duracao": duracao, "query": query, "dsParametro": dsParametro, "execUrl": execUrl}

        fn_LogSucceeded(parametros, dt_ingestao.strftime(format_log))

        return f"Carga da tabela {param.ds_Nome_Arquivo_Landing} finalizada com sucesso."

    except Exception as error:
        ## LOG ERRO
        dtFim = datetime.today() - timedelta(hours=3)
        dtFim_format = dtFim.strftime(format_log)
        duracao = int((dtFim-dtInicio).total_seconds()) #captura a duração subtraindo o dtFim pelo dtInicio
        dsParametro = str(param.asDict()) #captura todos os parâmetros
        notebook = dbutils.notebook.entry_point.getDbutils().notebook().getContext().notebookPath().get()
        try:
            pipeline = param.nm_Pipeline
        except:
            pipeline = os.path.basename(notebook)

        if end_point_page is not None:
            query = end_point_page
        else:
            query = ' '

        if hasattr(error, 'code'): # captura o código do erro, caso possua o atributo 'code'
            error_code = error.code
        else:
            error_code = 'NULL'
            
        parametros = {"tipo_log": tipo_log,"id_parametro": param.id_Parametro_Carga_API, "camada": camada, "dtInicio": dtInicio_format, "dtFim": dtFim_format, "pipeline": pipeline, "atividade": atividade, "notebook": notebook, "origem": origem, "destino": destino, "sistema": param.nm_Sistema, "emissor": emissor, "duracao": duracao, "query": query, "dsParametro": dsParametro, "cd_erro": error_code, "erro": str(error), "execUrl": execUrl}

        fn_LogFailed(parametros, dt_ingestao.strftime(format_log))

In [ ]:
# Cria uma lista para armazenar todas as tarefas
tasks = []

# Cria uma instância do ThreadPoolExecutor com threads definidas (max_workers)
with concurrent.futures.ThreadPoolExecutor(max_workers=threads) as executor:
    # Percorre todas as pastas do diretório de origem
    for row in data_param:
        # Executa a função fn_StreamFromFolder_csv em uma thread do ThreadPoolExecutor
        task = executor.submit(get_tables, *(row, baseUrl, dt_inicio_unix, dt_fim, location_landing))
                               
        # Adiciona a tarefa à lista de tarefas
        tasks.append(task)

# Aguarda a conclusão de todas as tarefas
_ = concurrent.futures.wait(tasks, return_when='ALL_COMPLETED')

In [ ]:
for task in tasks:
    try:
        print(task.result(),'\n')
    except Exception as error:
        print(error)
        pass